In [ ]:
import pandas as pd

# Load cleaned dataset
cleaned_data_path = "cleaned_app_train2.csv"
app_train_cleaned = pd.read_csv(cleaned_data_path)
app_train_cleaned

,COMMONAREA_AVG,LIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_AVG,COMMONAREA_MODE,LIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_MODE,COMMONAREA_MEDI,LIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MEDI,CNT_CHILDREN,...,TARGET,COMMONAREA_AVG_MISSING,LIVINGAPARTMENTS_AVG_MISSING,NONLIVINGAPARTMENTS_AVG_MISSING,COMMONAREA_MODE_MISSING,LIVINGAPARTMENTS_MODE_MISSING,NONLIVINGAPARTMENTS_MODE_MISSING,COMMONAREA_MEDI_MISSING,LIVINGAPARTMENTS_MEDI_MISSING,NONLIVINGAPARTMENTS_MEDI_MISSING
0,0.0143,0.0202,0.0000,0.0144,0.0220,0.0,0.0144,0.0205,0.0000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0605,0.0773,0.0039,0.0497,0.0790,0.0,0.0608,0.0787,0.0039,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1445,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,3.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1446,0.0666,0.0841,0.0000,0.0673,0.0918,0.0,0.0671,0.0855,0.0000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1447,0.0127,0.0134,0.0000,0.0128,0.0147,0.0,0.0128,0.0137,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Cek jumlah baris duplikat
duplicate_count = app_train_cleaned.duplicated().sum()
print(f"🔹 Jumlah baris duplikat: {duplicate_count}")

if duplicate_count == 0:
    print("✅ Tidak ada baris duplikat.")
else:
    print("⚠️ Masih ada duplikat, perlu dihapus.")


🔹 Jumlah baris duplikat: 0
✅ Tidak ada baris duplikat.


In [ ]:
# ==============================================
# PART 2: FEATURE ENGINEERING & DATA INTEGRATION
# ==============================================
import pandas as pd
import numpy as np

print("📂 Loading fully cleaned dataset from Part 1...")
app_train_cleaned = pd.read_csv("cleaned_app_train2.csv")
print(f"✅ Dataset loaded: {app_train_cleaned.shape}")
print(f"🔹 Missing values: {app_train_cleaned.isnull().sum().sum()}")
print(f"🔹 Duplicates: {app_train_cleaned.duplicated().sum()}")
print("\n🔧 Starting Feature Engineering...")

📂 Loading fully cleaned dataset from Part 1...
✅ Dataset loaded: (1449, 106)
🔹 Missing values: 75
🔹 Duplicates: 0

🔧 Starting Feature Engineering...


In [ ]:

# ==============================================
# FEATURE ENGINEERING (MAIN TABLE)
# ==============================================
# 1. Financial Health Ratios (MOST IMPORTANT)
app_train_cleaned['PAYMENT_TO_INCOME_RATIO'] = app_train_cleaned['AMT_ANNUITY'] / app_train_cleaned['AMT_INCOME_TOTAL']
app_train_cleaned['LOAN_TO_VALUE_RATIO'] = app_train_cleaned['AMT_CREDIT'] / app_train_cleaned['AMT_GOODS_PRICE']
app_train_cleaned['INCOME_ADEQUACY_RATIO'] = app_train_cleaned['AMT_ANNUITY'] / (app_train_cleaned['AMT_INCOME_TOTAL'] / 12)

# Handle division by zero or missing values for these new columns
ratio_cols = ['PAYMENT_TO_INCOME_RATIO', 'LOAN_TO_VALUE_RATIO', 'INCOME_ADEQUACY_RATIO']
for col in ratio_cols:
    app_train_cleaned[col] = app_train_cleaned[col].replace([np.inf, -np.inf], np.nan)
    app_train_cleaned[col] = app_train_cleaned[col].fillna(app_train_cleaned[col].median())
# 2. Boolean Flags from Categorical Variables
app_train_cleaned['FLAG_HAS_CAR'] = (app_train_cleaned['FLAG_OWN_CAR'] == 'Y').astype(int)
app_train_cleaned['FLAG_HAS_PROPERTY'] = (app_train_cleaned['FLAG_OWN_REALTY'] == 'Y').astype(int)

# 3. Interaction Features
app_train_cleaned['INTERACTION_GENDER_PROPERTY'] = app_train_cleaned['CODE_GENDER'] + '_' + app_train_cleaned['FLAG_OWN_REALTY']

print("✅ Main application table features created!")
print(f"New shape: {app_train_cleaned.shape}")

# ==============================================
# SELANJUTNYA: LOAD & ENGINEER SUPPORTING TABLES
# ==============================================

supporting_tables = [
    'bureau.csv',
    'previous_application.csv',
    'installments_payments.csv',
    'bureau_balance.csv',
    'POS_CASH_balance.csv',
    'credit_card_balance.csv'
]

tables = {}
for table_name in supporting_tables:
    try:
        df_temp = pd.read_csv(table_name)
        key_name = table_name.replace('.csv','')
        tables[key_name] = df_temp
        print(f"✅ {table_name} loaded: {df_temp.shape}, missing: {df_temp.isnull().sum().sum()}")

        # Basic cleaning for supporting tables (MASIH DIBUTUHKAN)
        num_cols_table = df_temp.select_dtypes(include=['float64','int64']).columns.tolist()
        cat_cols_table = df_temp.select_dtypes(include=['object']).columns.tolist()

        if len(num_cols_table) > 0:
            df_temp[num_cols_table] = df_temp[num_cols_table].fillna(df_temp[num_cols_table].median())
        if len(cat_cols_table) > 0:
            df_temp[cat_cols_table] = df_temp[cat_cols_table].fillna('Unknown')

        tables[key_name] = df_temp

    except FileNotFoundError:
        print(f"⚠️  {table_name} not found, skipping...")

✅ Main application table features created!
New shape: (1449, 112)
✅ bureau.csv loaded: (74390, 17), missing: 176426
✅ previous_application.csv loaded: (89619, 37), missing: 575075
✅ installments_payments.csv loaded: (98649, 8), missing: 3
✅ bureau_balance.csv loaded: (381526, 3), missing: 2
✅ POS_CASH_balance.csv loaded: (186886, 8), missing: 872
✅ credit_card_balance.csv loaded: (47578, 23), missing: 75700


In [ ]:
# ==============================================
# Langkah 4.1: Feature Engineering - Bureau
# ==============================================

import pandas as pd
import numpy as np

# Ambil tabel bureau dari dictionary tables
bureau = tables['bureau']

print(f"📊 Original bureau shape: {bureau.shape}")
print(f"🔹 Missing values: {bureau.isnull().sum().sum()}")

# Pilih kolom numeric yang akan diaggregasi (exclude SK_IDs)
bureau_cols = [col for col in bureau.columns if col not in ['SK_ID_CURR','SK_ID_BUREAU']][:10]

# Fungsi agregasi
def create_agg_features(df, groupby_col, agg_cols, prefix):
    agg_dict = {}
    for col in agg_cols:
        if df[col].dtype in ['float64','int64']:
            agg_dict[col] = ['mean','sum','max','min','std','count']
        else:
            agg_dict[col] = ['count','nunique']
    agg_df = df.groupby(groupby_col).agg(agg_dict).reset_index()
    # Flatten multi-level columns
    agg_df.columns = [groupby_col if col[0]==groupby_col else f"{prefix}_{col[0]}_{col[1]}"
                      for col in agg_df.columns]
    return agg_df

# Agregasi numeric/general columns
bureau_agg = create_agg_features(bureau, 'SK_ID_CURR', bureau_cols, 'BUREAU')

# Fitur tambahan khusus bisnis
bureau_add = bureau.groupby('SK_ID_CURR').agg({
    'DAYS_CREDIT': ['min','max','mean'],
    'CREDIT_DAY_OVERDUE': ['max','mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['max','mean'],
    'CREDIT_ACTIVE': ['count','nunique'],
    'CREDIT_TYPE': ['count','nunique']
}).reset_index()

# Flatten column names
bureau_add.columns = ['SK_ID_CURR'] + [f"BUREAU_{col[0]}_{col[1]}" for col in bureau_add.columns[1:]]

# [TAMBAHKAN CODE INI] Gabungkan fitur bureau_agg dan bureau_add
bureau_features = bureau_agg.merge(bureau_add, on='SK_ID_CURR', how='outer')
print(f"✅ Bureau features merged. Final shape: {bureau_features.shape}")
print(f"🔹 Missing values in bureau_features: {bureau_features.isnull().sum().sum()}")

📊 Original bureau shape: (74390, 17)
🔹 Missing values: 0
✅ Bureau features merged. Final shape: (15084, 64)
🔹 Missing values in bureau_features: 18360


In [ ]:
# ==============================================
# HANDLE MISSING VALUES IN BUREAU FEATURES
# ==============================================
print("🧹 Handling missing values in bureau features...")

# Untuk numerical features, isi dengan median
num_cols_bureau = bureau_features.select_dtypes(include=['float64', 'int64']).columns.tolist()
num_cols_bureau = [col for col in num_cols_bureau if col != 'SK_ID_CURR']

for col in num_cols_bureau:
    bureau_features[col] = bureau_features[col].fillna(bureau_features[col].median())

print(f"✅ Missing values after cleaning: {bureau_features.isnull().sum().sum()}")

🧹 Handling missing values in bureau features...
✅ Missing values after cleaning: 0


In [ ]:
# ==============================================
# Langkah 4.2: Feature Engineering - Previous Application
# ==============================================

prev_app = tables['previous_application']

# Pastikan kolom numeric sudah benar
num_cols_prev = ['AMT_ANNUITY','AMT_APPLICATION','AMT_CREDIT','AMT_DOWN_PAYMENT',
                 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START','RATE_DOWN_PAYMENT']
for col in num_cols_prev:
    if col in prev_app.columns:
        prev_app[col] = pd.to_numeric(prev_app[col], errors='coerce')  # ✅ Handle non-numeric
        prev_app[col] = prev_app[col].fillna(prev_app[col].median())   # ✅ Impute dengan median

# Pilih kolom untuk agregasi
prev_cols = [col for col in num_cols_prev if col in prev_app.columns]

# Agregasi numeric/general
prev_agg = create_agg_features(prev_app,'SK_ID_CURR',prev_cols,'PREV')

# Fitur tambahan khusus bisnis
prev_add = prev_app.groupby('SK_ID_CURR').agg({
    'NAME_CONTRACT_STATUS': ['count'],
    'CNT_PAYMENT': ['mean','sum','max'],
    'DAYS_DECISION': ['mean','min','max'],
    'DAYS_FIRST_DRAWING': ['mean']
}).reset_index()

# Flatten column names
prev_add.columns = ['SK_ID_CURR'] + [f"PREV_{col[0]}_{col[1]}" for col in prev_add.columns[1:]]

# Approval rate
if 'NAME_CONTRACT_STATUS' in prev_app.columns:
    approval_rate = prev_app.groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'] \
                           .apply(lambda x: (x=='Approved').mean()).reset_index()
    approval_rate.columns = ['SK_ID_CURR','PREV_APPROVAL_RATE']
    prev_add = prev_add.merge(approval_rate, on='SK_ID_CURR', how='left')

# Gabungkan fitur agregasi dan bisnis
prev_features = prev_agg.merge(prev_add, on='SK_ID_CURR', how='outer')

print(f"✅ Previous application features created: {prev_features.shape}")
print(f"🔹 Missing values in prev_features: {prev_features.isnull().sum().sum()}")


✅ Previous application features created: (73146, 52)
🔹 Missing values in prev_features: 419818


In [ ]:
# Handle missing values untuk numerical columns
num_cols_prev = prev_features.select_dtypes(include=['float64','int64']).columns.tolist()
num_cols_prev = [col for col in num_cols_prev if col != 'SK_ID_CURR']

for col in num_cols_prev:
    prev_features[col] = prev_features[col].fillna(prev_features[col].median())

print(f"✅ Missing values after cleaning: {prev_features.isnull().sum().sum()}")

✅ Missing values after cleaning: 0


In [ ]:
# ==============================================
# Langkah 4.3: Feature Engineering - Installment Payments
# ==============================================

install = tables['installments_payments']

# Pastikan kolom numeric
num_cols_install = ['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT',
                    'DAYS_ENTRY_PAYMENT','AMT_INSTALMENT','AMT_PAYMENT']
for col in num_cols_install:
    if col in install.columns:
        install[col] = pd.to_numeric(install[col], errors='coerce')

# Agregasi dasar
install_agg = install.groupby('SK_ID_CURR').agg({
    'NUM_INSTALMENT_VERSION':['max','mean'],
    'NUM_INSTALMENT_NUMBER':['max','mean'],
    'DAYS_INSTALMENT':['min','max','mean'],
    'DAYS_ENTRY_PAYMENT':['min','max','mean'],
    'AMT_INSTALMENT':['sum','mean','max','std'],
    'AMT_PAYMENT':['sum','mean','max','std']
}).reset_index()

# Flatten column names
install_agg.columns = ['SK_ID_CURR'] + [f"INSTAL_{col[0]}_{col[1]}" for col in install_agg.columns[1:]]

# [PERBAIKI] Fitur performa pembayaran (tanpa warning)
payment_perf = install.groupby('SK_ID_CURR', group_keys=False).apply(lambda x: pd.Series({
    'PAYMENT_PERF_MEAN': (x['AMT_PAYMENT']/x['AMT_INSTALMENT']).mean(),
    'PAYMENT_PERF_VAR': (x['AMT_PAYMENT']/x['AMT_INSTALMENT']).var(),
    'LATE_PAYMENT_COUNT': (x['DAYS_ENTRY_PAYMENT']>x['DAYS_INSTALMENT']).sum(),
    'EARLY_PAYMENT_COUNT': (x['DAYS_ENTRY_PAYMENT']<x['DAYS_INSTALMENT']).sum()
})).reset_index()

# [PERBAIKI] Handle infinite values (tanpa warning)
payment_perf['PAYMENT_PERF_MEAN'] = payment_perf['PAYMENT_PERF_MEAN'].replace([np.inf, -np.inf], np.nan)
payment_perf['PAYMENT_PERF_VAR'] = payment_perf['PAYMENT_PERF_VAR'].replace([np.inf, -np.inf], np.nan)

# Gabungkan
install_features = install_agg.merge(payment_perf, on='SK_ID_CURR', how='left')

print(f"✅ Installment payment features created: {install_features.shape}")
print(f"🔹 Missing values in install_features: {install_features.isnull().sum().sum()}")

✅ Installment payment features created: (48250, 23)
🔹 Missing values in install_features: 73175


/tmp/ipython-input-3319277630.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  payment_perf = install.groupby('SK_ID_CURR', group_keys=False).apply(lambda x: pd.Series({


In [ ]:
# Setelah install_features = install_agg.merge(payment_perf, on='SK_ID_CURR', how='left')

# Handle missing values untuk numerical columns
num_cols_install = install_features.select_dtypes(include=['float64','int64']).columns.tolist()
num_cols_install = [col for col in num_cols_install if col != 'SK_ID_CURR']

for col in num_cols_install:
    install_features[col] = install_features[col].fillna(install_features[col].median())

print(f"✅ Missing values after cleaning: {install_features.isnull().sum().sum()}")

✅ Missing values after cleaning: 0


In [ ]:
# [TAMBAHKAN SEBELUM MERGE] Validasi kunci
print("Validating key columns...")
print(f"SK_ID_CURR in app_train_cleaned: {'SK_ID_CURR' in app_train_cleaned.columns}")
print(f"SK_ID_CURR in bureau_features: {'SK_ID_CURR' in bureau_features.columns}")
print(f"SK_ID_CURR in prev_features: {'SK_ID_CURR' in prev_features.columns}")
print(f"SK_ID_CURR in install_features: {'SK_ID_CURR' in install_features.columns}")

Validating key columns...
SK_ID_CURR in app_train_cleaned: True
SK_ID_CURR in bureau_features: True
SK_ID_CURR in prev_features: True
SK_ID_CURR in install_features: True


In [ ]:
# ==============================================
# MERGE ALL FEATURES
# ==============================================
print("\n🔄 Merging all features to main dataset...")

app_train_fe = app_train_cleaned.copy()

# Merge Bureau features
app_train_fe = app_train_fe.merge(bureau_features, on='SK_ID_CURR', how='left')

# Merge Previous Application features
app_train_fe = app_train_fe.merge(prev_features, on='SK_ID_CURR', how='left')

# Merge Installment Payment features
app_train_fe = app_train_fe.merge(install_features, on='SK_ID_CURR', how='left')

print(f"🚀 Final dataset shape: {app_train_fe.shape}")
print(f"🔹 Missing values after merge: {app_train_fe.isnull().sum().sum()}")

# Final cleaning untuk missing values hasil merge
for col in app_train_fe.columns:
    if app_train_fe[col].dtype in ['float64','int64']:
        app_train_fe[col] = app_train_fe[col].fillna(app_train_fe[col].median())
    elif app_train_fe[col].dtype == 'object':
        app_train_fe[col] = app_train_fe[col].fillna('Unknown')

print(f"✅ Final missing values: {app_train_fe.isnull().sum().sum()}")


🔄 Merging all features to main dataset...
🚀 Final dataset shape: (1449, 248)
🔹 Missing values after merge: 161442
✅ Final missing values: 0


In [ ]:
 #[TAMBAHKAN SETELAH MERGE] Untuk memastikan merge bekerja
print(f"Shape sebelum merge: {app_train_cleaned.shape}")
print(f"Shape setelah merge: {app_train_fe.shape}")
print(f"Jumlah baris yang ditambahkan: {app_train_fe.shape[0] - app_train_cleaned.shape[0]}")

Shape sebelum merge: (1449, 112)
Shape setelah merge: (1449, 248)
Jumlah baris yang ditambahkan: 0


In [ ]:
# Cek semua kolom yang ada
print(app_train_fe.columns.tolist())

# Cek khusus kolom yang seharusnya sudah Anda buat
new_features_to_check = [
    'PAYMENT_TO_INCOME_RATIO',
    'LOAN_TO_VALUE_RATIO',
    'INCOME_ADEQUACY_RATIO',
    'FLAG_HAS_CAR',
    'FLAG_HAS_PROPERTY',
    'INTERACTION_GENDER_PROPERTY',
    'BUREAU_DAYS_CREDIT_mean_x',# Contoh fitur dari bureau_agg
    'BUREAU_DAYS_CREDIT_mean_y',# Contoh fitur dari bureau_add
    'PREV_APPROVAL_RATE',      # Contoh fitur dari previous app
    'PAYMENT_PERF_MEAN'        # Contoh fitur dari installments
]

for feature in new_features_to_check:
    if feature in app_train_fe.columns:
        print(f"✅ FOUND: {feature}")
    else:
        print(f"❌ MISSING: {feature}")

['COMMONAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'COMMONAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_MODE', 'COMMONAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAR

In [ ]:
# ==============================================
# 7️⃣ Simpan dataset final
# ==============================================
final_dataset_path = "app_train_with_features_cleaned1.csv"
app_train_fe.to_csv(final_dataset_path, index=False)
print(f"✅ Dataset final tersimpan di '{final_dataset_path}'")


✅ Dataset final tersimpan di 'app_train_with_features_cleaned1.csv'


In [ ]:
# Final verification sebelum save
print("\n🔍 Final dataset verification:")
print(f"Shape: {app_train_fe.shape}")
print(f"Missing values: {app_train_fe.isnull().sum().sum()}")
print(f"Duplicates: {app_train_fe.duplicated().sum()}")
print(f"Target distribution:\n{app_train_fe['TARGET'].value_counts(normalize=True)}")

# Pastikan SK_ID_CURR masih ada untuk referensi
if 'SK_ID_CURR' in app_train_fe.columns:
    print(f"Unique customers: {app_train_fe['SK_ID_CURR'].nunique()}")


🔍 Final dataset verification:
Shape: (1449, 248)
Missing values: 0
Duplicates: 0
Target distribution:
TARGET
0.0    0.929607
1.0    0.070393
Name: proportion, dtype: float64
Unique customers: 1449
